In [35]:
import pandas as pd

In [36]:
Label_df = pd.read_csv('egd_Labels.txt',names=['ID','label','extra'],sep='\t', dtype={'ID':str,'label':int,'extra':int})
df = pd.read_csv('FullyAnnotatedAndTrainTestSplitReady_egd.csv')

In [37]:
df.head()

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path
0,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,data/egd/NIFTI/MR_20020205/20020205/MR/1.2.826...
1,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/8-T2_...,8-T2_Ax,T2w,MR_20020205_EGD-0924,EGD-0924,False,data/egd/NIFTI/MR_20020205/20020205/MR/1.2.826...
2,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/3-3D_...,3-3D_Ax_T1_FSPGR_HR_blanco,T1w,MR_20150326_EGD-1215,EGD-1215,True,data/egd/NIFTI/MR_61373/20170605/MR/1.2.826.0....
3,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/650-a...,650-ax_cube,pwi,MR_20150326_EGD-1215,EGD-1215,True,data/egd/NIFTI/MR_61373/20170605/MR/1.2.826.0....
4,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/450-A...,450-ADC__10__6_mm__s__Nov_02_2018_20_57_56_CET,adc,MR_20150326_EGD-1215,EGD-1215,True,data/egd/NIFTI/MR_61373/20170605/MR/1.2.826.0....


In [47]:
df.describe()

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path
count,2442,2442,2442,2442,2442,2442,2442,2442,2442
unique,2370,2361,2424,856,8,260,104,2,2370
top,data/egd/DICOM_STRUCTURED/MR_29988/20170615/MR...,1.2.826.0.1.3680043.10.104.1002580906157172357...,data/egd/data/MR_20100131_EGD-0381/scans/9-T1G...,4-DWI_DWI_Ax,pwi,MR_20150526_EGD-1220,EGD-0119,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_29...
freq,2,4,2,53,500,27,162,2120,2


In [38]:
def extractNiftiFilepathAndSlicenum(df):
    ID = df['ID']
    split = ID.rsplit('__s', 1)
    NiftiPath = split[0]
    NiftiPath = NiftiPath.replace('NIFTI_SLICES', 'NIFTI') + '.nii.gz'
    slicenum = int(split[1].split('.nii.gz')[0])
    return pd.Series({'NiftiPath': NiftiPath, 'slicenum': slicenum})

In [39]:
appendstring = '/trinity/home/r098375/DDS/'
def fixPath(ID):
    if not 'trinity' in ID:
        ID = appendstring+ID
    basepart = '/'.join((ID.split('/')[:9]))
    namepart = '__'.join((ID.split('/')[9:]))
    ID = basepart+'/'+namepart+'.nii.gz'
    return ID

In [40]:
df['NIFTI_path'] = df['NIFTI_path'].apply(fixPath)

In [46]:
df['NIFTI_path'][0]

'/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20020205/20020205__MR__1.2.826.0.1.3680043.10.104.2857427232523006061017867594417320946__1.2.826.0.1.3680043.10.104.1456467217459860599267812336880387893.nii.gz'

In [42]:
df_NS = Label_df.apply(extractNiftiFilepathAndSlicenum, axis=1, result_type='expand')
Label_df = Label_df.merge(df_NS, left_index=True, right_index=True, how='inner')

In [43]:
Label_df['NiftiPath'][0]

'/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20020205/20020205__MR__1.2.826.0.1.3680043.10.104.2857427232523006061017867594417320946__1.2.826.0.1.3680043.10.104.2282046845047365453865806177133221068.nii.gz'

In [44]:
df_new = pd.merge(df,Label_df,how='inner',right_on='NiftiPath',left_on='NIFTI_path')

In [45]:
df_new.head(100)

,structured_path,filename,path,raw_name,name,label_exp,subject,train,NIFTI_path,ID,label,extra,NiftiPath,slicenum
0,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,11
1,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,8
2,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,21
3,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,4
4,data/egd/DICOM_STRUCTURED/MR_20020205/20020205...,1.2.826.0.1.3680043.10.104.2857427232523006061...,data/egd/data/MR_20020205_EGD-0924/scans/3-T1_...,3-T1_3D,T1w,MR_20020205_EGD-0924,EGD-0924,False,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_20...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/450-A...,450-ADC__10__6_mm__s__Nov_02_2018_20_57_56_CET,adc,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,1
96,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/450-A...,450-ADC__10__6_mm__s__Nov_02_2018_20_57_56_CET,adc,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,21
97,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/450-A...,450-ADC__10__6_mm__s__Nov_02_2018_20_57_56_CET,adc,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,17
98,data/egd/DICOM_STRUCTURED/MR_61373/20170605/MR...,1.2.826.0.1.3680043.10.104.1044043140781656433...,data/egd/data/MR_20150326_EGD-1215/scans/450-A...,450-ADC__10__6_mm__s__Nov_02_2018_20_57_56_CET,adc,MR_20150326_EGD-1215,EGD-1215,True,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,/trinity/home/r098375/DDS/data/egd/NIFTI_SLICE...,0,0,/trinity/home/r098375/DDS/data/egd/NIFTI/MR_61...,13


In [48]:
df_new.describe()

,label,extra,slicenum
count,47950.0,47950.0,47950.000000
mean,0.0,0.0,12.000000
std,0.0,0.0,7.211178
min,0.0,0.0,0.000000
25%,0.0,0.0,6.000000
50%,0.0,0.0,12.000000
75%,0.0,0.0,18.000000
max,0.0,0.0,24.000000


In [50]:
Label_df.describe()

,label,extra,slicenum
count,81175.0,81175.0,81175.000000
mean,0.0,0.0,12.000000
std,0.0,0.0,7.211147
min,0.0,0.0,0.000000
25%,0.0,0.0,6.000000
50%,0.0,0.0,12.000000
75%,0.0,0.0,18.000000
max,0.0,0.0,24.000000


In [51]:
2442*25

61050